In [1]:
from discrete_motif import DiscreteGrnMotif
import discrete_motif_functions as functions
import discrete_motif_operations as operations
import discrete_motif_measures as measures
import discrete_motif_generator as generator
import discrete_motif_plotting as visualize
from IPython.display import HTML, display
import tabulate

# Example 1: X-OR

In this example, we demonstrate an X-OR motif, where gene $A_{t=0}$ and $B_{t=0}$ set $B_{t=1}$.

First, we we create an empty motif with 1 variable and 2 possible states.

In [ ]:
motif_xor = DiscreteGrnMotif(1, 2, 'random')

We then enter our motif by adding a second gene, defining a rule, setting the decision rule in case of conflicting rules, and then contructing the GRN-object.

In [ ]:
motif_xor.grn_vars["gene_cnt"] = 2
motif_xor.grn_vars["conflict_rule"] = 'totaleffect'
motif_xor.append_rule([0, 1], [1], functions.xor)
motif_xor.construct_grn()

We can now evaluate our motif, one timestep at a time. We do this once so we can look at results.

In [ ]:
motif_xor.evaluate_motif(genes=[0, 1])

We can look at marginalized probability distributions of the system by accessing the states list, which is appended to every time you run the evaluate_motif function. We can also look at the transition state table. Note that the X-OR determines the increase of gene 2, meaning that in the (1, 1) case we still get a 1 returned (namely 1 + 0).

In [ ]:
print("The states: ")
for state in motif_xor.states:
    print(state)
table = visualize.state_transition_table(motif_xor, 'totaleffect')
display(HTML(tabulate.tabulate(table, tablefmt='html')))

We can perform several analysis methods on this motif. We find that there is a significant amount of synergy in this system, as we expect. It is also about the amount we should find: there is 2 bits of information mutual information, and 1 bit synergy.

In [ ]:
print("The mutual information: ")
print(str(measures.mutual_information(motif_xor, [0]))+" component 1")
print(str(measures.mutual_information(motif_xor, [1]))+" component 2")
print(str(measures.mutual_information(motif_xor))+" total")
print("The WMS information: ")
print(measures.synergy_wms(motif_xor))
print("The SRV synergy: ")
print(measures.synergy_quax(motif_xor))
print("The UII synergy: ")
print(measures.synergy_uii(motif_xor))
print("The middle ground synergy approximation: ")
print(measures.synergy_middleground(motif_xor))

We can examine the decay of mutual information over multiple timesteps with a build-in function. In this case this is not very interesting.

In [ ]:
print(measures.mi_decay(motif_xor))

Finally, we can reset the state, nudge 2 genes with a 0.5-impact nudge. After doing so, we can assess the nudge impact by comparing the state after our initial timestep, with the state after nudging and moving ahead one timestep.

In [ ]:
motif_xor.reset_to_state(0)
operations.nudge_variable(motif_xor, 2, 0.5)
motif_xor.evaluate_motif()
print("The nudge impact: ")
print(measures.hellinger(motif_xor.states[1], motif_xor.states[-1]))

# Example 2: AND

We can perform the same steps with an AND-gate.

In [ ]:
motif_and = DiscreteGrnMotif(1, 2, 'random')
motif_and.grn_vars["gene_cnt"] = 2
motif_and.grn_vars["conflict_rule"] = 'totaleffect'
motif_and.append_rule([0, 1], [1], functions.plus_and)
motif_and.construct_grn()
motif_and.evaluate_motif(genes=[0, 1])
print("The states: ")
for state in motif_and.states:
    print(state)
table = visualize.state_transition_table(motif_and, 'totaleffect')
display(HTML(tabulate.tabulate(table, tablefmt='html')))

After initiating, we can do our synergy and nudge impact analysis.

In [ ]:
print("The mutual information: ")
print(measures.mutual_information(motif_and))
print("The WMS information: ")
print(measures.synergy_wms(motif_and))
print("The middle ground synergy approximation: ")
print(measures.synergy_middleground(motif_and))
print("The memory: ")
print(measures.mi_decay(motif_and))
motif_and.reset_to_state(0)
operations.nudge_variable(motif_and, 2, 0.5)
motif_and.evaluate_motif()
print("The nudge impact: ")
print(measures.hellinger(motif_and.states[1], motif_and.states[-1]))

# Example 3: random motifs

To do randomized experiments, we also support the generation of random motifs. These motifs incorporate random relationships between genes, drawn from a library of all possible relations in a GRN motif.

First, we generate a single random motif of 2 genes, with 2 relationships.

In [ ]:
motifs, _ = generator.generate_motifs(samplesize=2, no_nodes=2, indegree=1, conflict_rule='totaleffect')
motif_rand = motifs[0]
motif_rand.evaluate_motif(genes=[0, 1])
print("The states: ")
for state in motif_rand.states:
    print(state)
table = visualize.state_transition_table(motif_rand, 'totaleffect')
display(HTML(tabulate.tabulate(table, tablefmt='html')))

We then can do our usual analysis.

In [ ]:
print("The rules: ")
print(motif_rand.grn_vars["rules"])
print("The correlations: ")
print(motif_rand.grn_vars["correlations"])
print("The mutual information: ")
print(measures.mutual_information(motif_rand))
print("The WMS information: ")
print(measures.synergy_wms(motif_rand))
print("The middle ground synergy approximation: ")
print(measures.synergy_middleground(motif_rand))
print("The memory: ")
print(measures.mi_decay(motif_rand))
motif_rand.reset_to_state(0)
operations.nudge_variable(motif_rand, 2, 0.5)
motif_rand.evaluate_motif()
print("The nudge impact: ")
print(measures.hellinger(motif_rand.states[1], motif_rand.states[-1]))

# Example 4: multivalued logic (3 levels)

The SRV measure should work better for higher-valued logic, we try this out too. We should check the correlation matrix generated, this should be something sensible.

In [2]:
motifs, _ = generator.generate_motifs(samplesize=1, no_nodes=3, indegree=3, conflict_rule='totaleffect', numvalues=3)
motif_rand = motifs[0]
motif_rand.evaluate_motif()
print("The states: ")
for state in motif_rand.states:
    print(state)
print("The correlations: ")
print(motif_rand.grn_vars["correlations"])

table = visualize.state_transition_table(motif_rand, 'totaleffect')
display(HTML(tabulate.tabulate(table, tablefmt='html')))

The states: 
[[[ 0.02302128  0.0266968   0.00887236]
  [ 0.01322066  0.01396102  0.02531221]
  [ 0.04703992  0.05937952  0.02547019]]

 [[ 0.05597212  0.03755121  0.05954974]
  [ 0.05922233  0.06046341  0.05508478]
  [ 0.03146739  0.0531337   0.03031707]]

 [[ 0.02230604  0.04571421  0.05537438]
  [ 0.04812822  0.00760127  0.0080223 ]
  [ 0.04004123  0.04555277  0.0415239 ]]]
[[[ 0.20217628  0.0  0.0]
  [ 0.12379647  0.0  0.0]
  [ 0.0  0.0  0.0]]

 [[ 0.0  0.069345956  0.0]
  [ 0.0  0.0  0.0]
  [ 0.0  0.0  0.0]]

 [[ 0.0  0.0  0.17885915]
  [ 0.0  0.0  0.0]
  [ 0.0  0.24009168  0.18573046]]]
The correlations: 
[[  0.00000000e+000  -4.99993801e-001   2.46811425e-003]
 [ -4.99993801e-001   6.92745162e-310  -4.93628969e-003]
 [  2.46811425e-003  -4.93628969e-003   6.92745835e-310]]


Gene 0 t=0,Gene 1 t=0,Gene 2 t=0,Gene 0 t=1,Gene 1 t=1,Gene 2 t=1
0,0,0,0,0,0
0,0,1,0,0,0
0,0,2,0,0,0
0,1,0,0,0,1
0,1,1,2,2,0
0,1,2,2,2,0
0,2,0,1,1,2
0,2,1,2,2,1
0,2,2,2,2,0
1,0,0,0,0,0


We then can do our usual analysis.

In [ ]:
print("The rules: ")
print(motif_rand.grn_vars["rules"])
print("The mutual information: ")
print(measures.mutual_information(motif_rand))
print("The WMS information: ")
print(measures.synergy_wms(motif_rand))
print("The middle ground synergy approximation: ")
print(measures.synergy_middleground(motif_rand))
print("The memory: ")
print(measures.mi_decay(motif_rand))
motif_rand.reset_to_state(0)
operations.nudge_variable(motif_rand, 2, 0.5)
motif_rand.evaluate_motif()
print("The nudge impact: ")
print(measures.hellinger(motif_rand.states[1], motif_rand.states[-1]))

The SRV synergy should be easier to compute in this case.

In [ ]:
print("The SRV synergy: ")
print(measures.synergy_quax(motif_rand))